### Векторные представления слов (word embeddings)

Векторное представление слова (word embedding) — вещественный вектор в пространстве с фиксированной невысокой размерностью.

                                            Пример векторных представлений слов (2D t-SNE)
<img src='image/2D_tsne.PNG'>

Зачем нужны Word embeddings?
Сжатые векторные представления слов
1. полезны сами по себе, например, для поиска
синонимов или опечаток в поисковых запросах.
2. используются в качестве признаков для решения
самых различных задач: выявление именованных сущностей, тэгирование частей речи, машинный перевод, кластеризация документов, ранжирование документов, анализ тональности текста.

#### word2vec

Мера семантической близости — мера близости, предназначенная для количественной оценки семантической схожести слов. Такая мера показывает высокие значения для пар слов, находящихся в семантических отношениях (синонимия, ассоциативность и т.д.), и нулевые значения для всех остальных пар.

word2vec - алгоритм для получения векторных представлений слов. Подход основан на важной гипотезе, которую в науке принято называть гипотезой локальности — “слова, которые встречаются в одинаковых окружениях, имеют близкие значения”. Близость в данном случае понимается очень широко, как то, что рядом могут стоять только сочетающиеся слова. Например, для нас привычно словосочетание "заводной будильник". А сказать “заводной апельсин” мы не можем* — эти слова не сочетаются.

##### Алгоритм word2vec
Мы будем предсказывать вероятность слова по его окружению (контексту). То есть мы будем учить такие вектора слов, чтобы вероятность, присваиваемая моделью слову была близка к вероятности встретить это слово в этом окружении в реальном тексте.





<img src='image/w2v_formula.PNG'>

Здесь W0 — вектор целевого слова, Wc — это некоторый вектор контекста, вычисленный (например, путем усреднения) из векторов окружающих нужное слово других слов. А S — это функция, которая двум векторам сопоставляет одно число. Например, это может быть косинусное расстояние.

Процесс тренировки устроен следующим образом: мы берем последовательно (2k+1) слов, слово в центре является тем словом, которое должно быть предсказано. А окружающие слова являются контекстом длины по k с каждой стороны. Каждому слову в нашей модели сопоставлен уникальный вектор, который мы меняем в процессе обучения нашей модели. В целом, этот подход называется CBOW — continuous bag of words, continuous потому, что мы скармливаем нашей модели последовательно наборы слов из текста, a BoW потому что порядок слов в контексте не важен.
<img src='image/CBOW_.png'>
Другой подход skip-gram — прямо противоположный CBOW, то есть “словосочетание с пропуском”. Мы пытаемся из данного нам слова угадать его контекст (точнее вектор контекста). В остальном модель не претерпевает изменений.
<img src='image/skipgram.png'>

Что стоит отметить: хотя в модель не заложено явно никакой семантики, а только статистические свойства корпусов текстов, оказывается, что натренированная модель word2vec может улавливать некоторые семантические свойства слов. Классический пример:

<img src='image/word_embeddings.PNG'>

Слово "мужчина" относится к слову "женщина" так же, как слово "дядя" к слову "тётя", что для нас совершенно естественно и понятно, но в других моделям добиться такого же соотношения векторов можно только с помощью специальных ухищрений. Здесь же — это происходит естественно из самого корпуса текстов.

#### Что мы можем попробовать сделать с векторами слов?

Мы можем делать различные синтаксические, семантические NLP задачи с векторами слов, некоторое из них уже встроены. 


In [1]:
import gensim.downloader as api

/home/rzaharov@mvs.local/GB/venv_course/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [2]:
import gensim.downloader as api

word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [5]:
# выведем лишнее слово
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

print(word_vectors.doesnt_match("black green summer brown".split()))

cereal
summer


In [6]:
# определим схожесть между словами
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

0.8323495
0.5288512
0.21199903


In [7]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("mouth", topn=3)
print(result)

[('woman', 0.832349419593811), ('boy', 0.7914870977401733), ('one', 0.7788748741149902)]
[('dog', 0.8798074722290039), ('rabbit', 0.7424426674842834), ('cats', 0.7323004007339478)]
[('tongue', 0.7366125583648682), ('mouths', 0.687748908996582), ('ear', 0.6811771988868713)]


# Simple chat-bot example

In [1]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

/home/rzaharov@mvs.local/GB/venv_course/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [11]:
!ls

187.zip  image		 lesson_2_v01.ipynb  lesson-2.zip  prepared_answers.txt
corpus	 lesson_2.ipynb  lesson_2_v02.ipynb  Otvety.txt


In [10]:
!head -5 prepared_answers.txt

	вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . 
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) .	меня вобще прикалывает эта тема :). 
Что делать, сегодня нашёл 2 миллиона рублей? .	Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. 
Эбу в двенашке называется Итэлма что за эбу? .	ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он принимает информацию от мно

In [3]:
counter_all = 0
counter_filter = 0
with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        counter_all +=1
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue
            
        counter_filter +=1

<ipython-input-3-73322b7b2d68>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


In [6]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-6-4e0de9aa106b>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


KeyboardInterrupt: 

In [2]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [8]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

<ipython-input-8-4964f3fb6446>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [9]:
sentences = [i for i in sentences if len(i) > 2]

In [10]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [13]:
Word2Vec?

In [11]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [14]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [16]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

<ipython-input-16-64550af31b99>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


True

In [25]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [32]:
TEXT = "какой город самы красивый"

In [33]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['<br> <br>. \n',
 'жить в родных краях, кататься по всему миру. \n',
 'Конечно. Это город в Дагестане на узком проходе между Каспийским морем и предгорьями Кавказа. Дербент — самый южный город Российской Федерации. <br>Дербент — иногда считают одним из древнейших «живущих» городов мира, старейшим городом Российской Федерации. Первые поселения возникли здесь в эпоху ранней бронзы — в конце 4 тысячелетия до н. э. . Первое упоминание Каспийских ворот — наиболее древнего названия Дербента — относится к VI в. до н. э. , его приводит известный древнегреческий географ Гекатей Милетский.. \n']

In [34]:
get_response(TEXT, ft_index, modelFT, index_map)

['г Санкт-Петербург.. \n',
 'Шеньчжень <br> <br> <br> <br>Город расположен в 120 км от Гуанчжоу. Одна из первых Специальных Экономических Зон Китая. Шэньчжэнь возник на месте бывшей рыбацкой деревушки на границе с Гонконгом. За 20 лет превратился в современный город с населением в 3 млн. человек. Въезд китайцев в Шэньчжэнь ограничен. В городе самый высокий уровень жизни в Китае. В городе и пригородах находятся предприятия Гонконга, Тайваня, Сингапура, Японии, Южной Кореи, США, стран Западной Европы. Свыше 80% производимой продукции экспортируется. <br> <br>http://travel.gala.net/ref/cn/_enchzhen/ <br>Многие иностранные посетители говорят, что Шэньчжэнь - это то, что было бы с Китаем, если бы не коммунизм. Шэнчжэнь является горячей туристической точкой в Китае. Здесь возможно прекрасно провести отпуск: пляжи, тематические парки, отличные отели. Но настоящая жизнь в Шеньчжене начинается ночью. Культурной жизнью города заправляют частные компании, чем объясняется необыкновенное разнообраз